In [51]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np
import datetime as dt

In [5]:
!pip install pygal

In [52]:
import pygal

In [53]:
dataFrame = pd.read_csv("contact-tracing.csv")
dataFrame.head()

,timestamp,latitude,longitude,name
0,1593373946,12.880172,77.784798,Alice
1,1593377546,12.993111,77.596512,Alice
2,1593381146,12.975756,77.463615,Alice
3,1593384746,12.974606,77.615150,Alice
4,1593388346,12.997514,77.706261,Alice


In [54]:
dataFrame["name"].value_counts()

Bob      25
Alice    24
Carol    24
David    24
Erin     24
Frank    24
Grace    24
Heidi    24
Ivan     24
Judy     24
Name: name, dtype: int64

In [55]:
from IPython.display import display, HTML
base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""


In [10]:
!pip install matplotlib

NameError: name 'dataFrame' is not defined

In [56]:
safe_distance = 0.0018288 # a radial distance of 6 feet in kilometers
model = DBSCAN(eps=safe_distance, min_samples=2, metric='haversine').fit(dataFrame[['latitude', 'longitude']])
core_samples_mask = np.zeros_like(model.labels_, dtype=bool)
core_samples_mask[model.core_sample_indices_] = True
labels = model.labels_
dataFrame['Cluster'] = model.labels_.tolist()

In [57]:
disp_dict = {}
for index, row in dataFrame.iterrows():
    if row['name'] not in disp_dict.keys():
        disp_dict[row['name']] = [(row['latitude'], row['longitude'])]
    else:
        disp_dict[row['name']].append((row['latitude'], row['longitude']))
xy_chart = pygal.XY(stroke=False)
[xy_chart.add(k,v) for k,v in sorted(disp_dict.items())]
display(HTML(base_html.format(rendered_chart=xy_chart.render(is_unicode=True))))

In [58]:
core_samples_mask

array([False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False,  True, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,

In [64]:
disp_dict_clust = {}
for index, row in dataFrame.iterrows():
    if row['Cluster'] not in disp_dict_clust.keys():
        disp_dict_clust[row['Cluster']] = [(row['latitude'], row['longitude'])]
    else:
        disp_dict_clust[row['Cluster']].append((row['latitude'], row['longitude']))
print(len(disp_dict_clust.keys()))
from pygal.style import LightenStyle
dark_lighten_style = LightenStyle('#F35548')
xy_chart = pygal.XY(stroke=False, style=dark_lighten_style)
[xy_chart.add(str(k),v) for k,v in disp_dict_clust.items()]
display(HTML(base_html.format(rendered_chart=xy_chart.render(is_unicode=True))))



30


In [60]:
dataFrame["Cluster"].value_counts()

-1     182
 1       3
 15      2
 27      2
 26      2
 25      2
 24      2
 23      2
 22      2
 21      2
 20      2
 19      2
 18      2
 17      2
 16      2
 14      2
 0       2
 13      2
 12      2
 11      2
 10      2
 9       2
 8       2
 7       2
 6       2
 5       2
 4       2
 3       2
 2       2
 28      2
Name: Cluster, dtype: int64

In [65]:
inputName = "Frank"
inputNameClusters = set()
for i in range(len(dataFrame)):
    if dataFrame['name'][i] == inputName:
        inputNameClusters.add(dataFrame['Cluster'][i])

In [66]:
infected = set()
for cluster in inputNameClusters:
    if cluster != -1:
            namesInCluster = dataFrame.loc[dataFrame['Cluster'] == cluster, 'name']
            for i in range(len(namesInCluster)):
                name = namesInCluster.iloc[i]
                if name != inputName:
                    infected.add(name)

In [67]:
print("Potential infections are:",*infected,sep="\n" )

Potential infections are:
Carol
Grace
